# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [ ]:
# google colab 用なのでコメントアウト
# !pip install tiktoken==0.7.0

In [1]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

Chat
GPT


### Tokenizer と tiktoken の紹介


In [2]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

37


## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [ ]:
# google colab 用なのでコメントアウト
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
from dotenv import load_dotenv

# 事前に .env ファイルを作って、OPENAI_API_KEY に API キーを設定してください
load_dotenv()

True

## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


In [ ]:
# !pip install openai==1.40.6

### Chat Completions API の呼び出し


In [4]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ALSGQWgDcUrVO2mwfVg15ZX612s4g",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことについてお話ししたいですか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729676898,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_482c22a7bc",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 25,
    "total_tokens": 53,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  }
}


### 会話履歴を踏まえた応答を得る


In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {
            "role": "assistant",
            "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？",
        },
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ALSGWZxDk0yAzmUlx3Lf60lOhBKPS",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前はジョンさんですね。何か特別なことについてお話ししたいことがありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729676904,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_482c22a7bc",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 69,
    "total_tokens": 96,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  }
}


### ストリーミングで応答を得る


In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？

### JSON モード


In [7]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


### Vision（画像入力）


In [8]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は書籍の表紙を示しており、タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」となっています。著者は吉田真吾氏と大嶋勇樹氏です。表紙のデザインには、緑色の鳥のイラストがあり、書籍名の「ChatGPT」の部分は際立ったデザインになっています。内容は大規模言語モデルを活用したシステム構築に関するもので、実践的な知識や技術を紹介していると思われます。


### （コラム）Completions API


In [9]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-ALSGoXclOm29GkWWgpTLAFEDb9W3N",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "私の出身はアメリカで、現在日本"
    }
  ],
  "created": 1729676922,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 11,
    "total_tokens": 27
  }
}


## 2.6. Function calling


### Function calling のサンプルコード


In [10]:
import json


def get_current_temparature(location: str, unit: str = "fahrenheit") -> str:
    """get current wethear

    Args:
        location (str): location name. e.g., Tokyo, San Francisco, Paris
        unit (str, optional): temparatoure unit. Defaults to "fahrenheit".

    Returns:
        str:  json format which has "location", "temperature", and "unit". like {"location": "Tokyo", "temperature": "10", "unit": "celsius"}
    """
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temparature",
            "description": "Get the current temparature in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [12]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0.0,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ALSH876EnTXPsVqh5VvEo3P7tj2w8",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_kiGt4RO4B1jKjf8cvIPvxhtz",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
              "name": "get_current_temparature"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1729676942,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_a7d06e42a7",
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 85,
    "total_tokens": 109,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  }
}


In [13]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())
messages

[{'role': 'user', 'content': '東京の天気はどうですか？'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_kiGt4RO4B1jKjf8cvIPvxhtz',
    'function': {'arguments': '{"location":"Tokyo, Japan","unit":"celsius"}',
     'name': 'get_current_temparature'},
    'type': 'function'}]}]

In [14]:
available_functions = {
    "get_current_temparature": get_current_temparature,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": "celsius"}


In [15]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_kiGt4RO4B1jKjf8cvIPvxhtz",
        "function": {
          "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
          "name": "get_current_temparature"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_kiGt4RO4B1jKjf8cvIPvxhtz",
    "role": "tool",
    "name": "get_current_temparature",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": \"celsius\"}"
  }
]


In [16]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-ALSHYhWUz2byaFeA2AvNJb2mJ6P5r",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "現在、東京の気温は10度です。天気の詳細は、天気予報サイトやアプリをご確認ください。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729676968,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_45c6de4934",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 70,
    "total_tokens": 99,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  }
}
